In [ ]:
import os
import copy
import subprocess
import numpy as np 
import renom as rm
from renom_img.api.segmentation.fcn import FCN16s
from PIL import Image
import renom_img
from renom_img.api.utility.augmentation import Augmentation
from renom_img.api.utility.augmentation.process import *
from renom_img.api.utility.evaluate.segmentation import segmentation_iou, segmentation_precision, segmentation_recall
from datetime import datetime
from renom.cuda import set_cuda_active
set_cuda_active(True)

In [ ]:
os.mkdir('model_logs')
date = str(datetime.date(datetime.now()))

In [ ]:
class_map = ['background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 
             'bus', 'car', 'cat', 'chair', 'cow', 'diningtable',
             'dog', 'horse', 'motorbike', 'person', 'potted plant', 
             'sheep', 'sofa', 'train', 'tv/monitor', 'void']

In [ ]:
def create_list(filename, mode):
    ann=[]
    img_path=[]
    with open(filename,'r') as f:
        for line in f:
            line = line[:-1]
            
            if mode is 'train':
                line = "/mnt/research/dataset/VOCdevkit/sbdd/img/"+line+".jpg"
                img_path.append(line)
                line = line.replace('img','cls')   
                line = line.replace('jpg','png')
                ann.append(line)
                
            elif mode is 'val':
                line = "/mnt/research/dataset/VOCdevkit/VOC2011/JPEGImages/"+line+".jpg"
                img_path.append(line)
                line = line.replace('JPEGImages','SegmentationClass')   
                line = line.replace('jpg','png')
                ann.append(line)
            else:
                print('Error: Mode must be either train or val')
                break

    return img_path,ann

In [ ]:
train_image, train_annot = create_list("/mnt/research/dataset/VOCdevkit/sbdd/train.txt","train")
valid_image, valid_annot = create_list("/mnt/research/dataset/VOCdevkit/VOC2011/ImageSets/Segmentation/seg11valid.txt","val")

In [ ]:
model = FCN16s(class_map, imsize=(224, 224),load_pretrained_weight=True, train_whole_network=True)

In [ ]:
aug = Augmentation([
                Flip(),
                WhiteNoise(),
                ContrastNorm([0.5, 1.0])
            ])

In [ ]:
def resize_annot(path_list,size):
    targets = []
    for path in path_list:
        annot = Image.open(path)
        annot.load()
        annot = np.array(annot.resize(size))
        annot[annot==255] = 21
    targets.append(annot)
    return np.array(targets)

In [ ]:
def end_function(*args):
    if len(args)>0:
#         calculating mAP
        model = args[1]
#         saving model
        train_list = args[2]
        validation_loss_list = args[3]
        epoch = args[0]
        if len(validation_loss_list)>1:
            tmp = copy.deepcopy(validation_loss_list)
            current_loss = tmp[-1]
            del(tmp[-1])
            tmp.sort()
            if(current_loss<tmp[0]):
                results = model.predict(valid_image)
                results = np.array(results)
                targets = resize_annot(valid_annot, model.imsize)
                precision, mean_precision = segmentation_precision(results, targets, ignore_class=[0,21])
                iou, mean_iou = segmentation_iou(results, targets, ignore_class=[0,21])
                recall, mean_recall = segmentation_recall(results, targets, ignore_class=[0,21])
                
                fp = open('model_logs/fcn16s@'+date+'.txt','a+')
                fp.write('Epoch: {:03d} Train Loss: {:3.2f}  Valid Loss: {:3.2f} mean_precision: {:3.2f} mean_iou: {:3.2f} mean_recall: {:3.2f} \n'.format(epoch,float(train_list[-1]),float(validation_loss_list[-1]),float(mean_precision),float(mean_iou),float(mean_recall)))
                fp.close()

In [ ]:
# Hyperparameters
total_epoch = 200
batch = 4
imsize = model.imsize
multiscale = None
optimizer = model._opt.__class__
augmentation = [str(name.__class__).split('.')[-1] for name in aug._process_list]
evaluation_matrix = "mean IoU"
dataset = "PASCAL_VOC_2011"
standard = 0.0
load_pretrained=True
train_whole=True
renom_v = rm.__version__
renom_img_v = renom_img.__version__
commit_id = str(subprocess.check_output(['git','rev-parse','HEAD']))

In [ ]:
# write hyperparameters to file
fp = open('model_logs/fcn16s@'+date+'.txt','a+')
fp.write('Commit Hash: '+commit_id[2:-3]+'\nReNom version: '+renom_v+'\nReNomIMG version: '+renom_img_v)
fp.write('\nExpected score: {:3.2f}\n'.format(float(standard)))
fp.write('\n===================================================Hyperparameters=======================================================\n')
fp.write('\nTotal epoch: {:03d}\nBatch size: {:03d}\nImage size: ({:03d},{:03d})'.format(total_epoch,batch,imsize[0],imsize[1]))
fp.write('\nMultiscale: '+str(multiscale)+'\nOptimizer: '+str(optimizer).split('.')[-1]+'\nAugmentation: '+str(augmentation))
fp.write('\nEvaluation matrix: '+str(evaluation_matrix)+'\nDataset: '+str(dataset))
fp.write('\nLoad Pretrained weight: '+str(load_pretrained)+'\nTrain whole network: '+str(train_whole))
fp.write('\n==========================================================================================================================\n\n')
fp.close()

In [ ]:
model.fit(train_img_path_list=train_image, train_annotation_list=train_annot,
          valid_img_path_list=valid_image, valid_annotation_list=valid_annot, 
          batch_size=batch, callback_end_epoch=end_function, augmentation=aug, epoch=total_epoch, class_weight=False)


fp = open('model_logs/fcn16s@'+date+'.txt','a')
fp.write('\nSuccess')
fp.close()